In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from skimage.io import imread
from skimage.transform import resize


In [62]:
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from skimage.io import imread
from skimage.transform import resize

def load_images_from_folder(folder):
    images = []
    labels = []
    class_names = []  # Added to store the mapping of class names to integer labels
    for class_folder in os.listdir(folder):
        class_path = os.path.join(folder, class_folder)
        if os.path.isdir(class_path):
            class_names.append(class_folder)
            for filename in os.listdir(class_path):
                img_path = os.path.join(class_path, filename)
                if img_path.endswith(".jpg"):
                    img = imread(img_path)
                    img = resize(img, (100, 100))
                    images.append(img.flatten())
                    labels.append(class_folder)  # Store class name as label
    return np.array(images), np.array(labels), class_names

train_folder = "/content/gdrive/MyDrive/class/train"
test_folder = "/content/gdrive/MyDrive/class/test"

X_train, y_train, class_names = load_images_from_folder(train_folder)
X_test, y_test, _ = load_images_from_folder(test_folder)  # Use the same class_names obtained from the training set

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

# Rest of the code (KMeans, PCA, SVM) remains the same


X_train shape: (1400, 30000)
y_train shape: (1400,)


In [63]:
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_train)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=2, random_state=42)

In [64]:
train_cluster_labels = kmeans.labels_
test_cluster_labels = kmeans.predict(X_test)


In [65]:
pca = PCA(n_components=50)  # Choose the number of components
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)


In [66]:
clf = svm.SVC()
clf.fit(X_train_pca, y_train)


SVC()

In [68]:
y_pred = clf.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.675


In [69]:
# Function to load and preprocess a single image
def load_and_preprocess_image(img_path):
    img = imread(img_path)
    img = resize(img, (100, 100))
    return img.flatten()

# Example path to a new image for prediction
new_image_path = "/content/gdrive/MyDrive/class/test/hamburger/100930.jpg"

# Load and preprocess the new image
new_image = load_and_preprocess_image(new_image_path)

# Reshape the image to match the shape of the training data
new_image = new_image.reshape(1, -1)

# Use the trained KMeans model to get cluster labels for the new image
new_image_cluster_label = kmeans.predict(new_image)

# Use the trained PCA model to transform the new image
new_image_pca = pca.transform(new_image)

# Use the trained SVM classifier to predict the label for the new image
new_image_pred = clf.predict(new_image_pca)

# Map cluster labels to class labels
class_label_mapping = {0: 'hamburger', 1: 'cheesecake'}

# Display the predicted cluster label and corresponding class label
predicted_cluster = new_image_cluster_label[0]
predicted_class = class_label_mapping[predicted_cluster]

print(f"Predicted Cluster Label: {predicted_cluster}")
print(f"Predicted Class Label: {predicted_class}")


Predicted Cluster Label: 0
Predicted Class Label: hamburger
